In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from random import randint
import time
import utils
import os
import pandas as pd

## Load Data

In [21]:
dataset_folder='/dataset_v1.0/'
train_folder = '/dataset_v1.0/train/'
test_folder = '/dataset_v1.0/test/'

train_size = len(os.listdir(train_folder))
test_size = len(os.listdir(test_folder))

train_size,test_size

(980, 250)

In [23]:
label_df = pd.read_csv(dataset_folder+'label.csv')
label_df.head()

,filename,breed,breed fac
0,affenpinscher_1.jpg,affenpinscher,0
1,affenpinscher_10.jpg,affenpinscher,0
2,affenpinscher_100.jpg,affenpinscher,0
3,affenpinscher_101.jpg,affenpinscher,0
4,affenpinscher_102.jpg,affenpinscher,0


In [20]:
train_image=torch.load(dataset_folder+'train.pt')
print(train_image)

tensor([[[[0.9412, 0.9412, 0.9373,  ..., 0.7961, 0.8000, 0.8039],
          [0.9294, 0.9294, 0.9333,  ..., 0.7412, 0.7451, 0.7451],
          [0.9216, 0.9216, 0.9255,  ..., 0.7255, 0.7255, 0.7255],
          ...,
          [0.8118, 0.8000, 0.7647,  ..., 0.6745, 0.6706, 0.6627],
          [0.7647, 0.7333, 0.6941,  ..., 0.6941, 0.6980, 0.6863],
          [0.7059, 0.6667, 0.6510,  ..., 0.7059, 0.7294, 0.7255]],

         [[0.9451, 0.9451, 0.9412,  ..., 0.8000, 0.8039, 0.8078],
          [0.9333, 0.9333, 0.9373,  ..., 0.7451, 0.7490, 0.7490],
          [0.9255, 0.9255, 0.9294,  ..., 0.7294, 0.7294, 0.7294],
          ...,
          [0.7569, 0.7569, 0.7216,  ..., 0.6510, 0.6431, 0.6353],
          [0.7098, 0.6784, 0.6510,  ..., 0.6706, 0.6706, 0.6588],
          [0.6510, 0.6118, 0.6078,  ..., 0.6824, 0.7020, 0.6980]],

         [[0.9647, 0.9647, 0.9608,  ..., 0.8157, 0.8196, 0.8235],
          [0.9529, 0.9529, 0.9569,  ..., 0.7608, 0.7647, 0.7647],
          [0.9451, 0.9451, 0.9490,  ..., 0

In [24]:
test_image=torch.load(dataset_folder+'test.pt')
print(train_image)

tensor([[[[0.9412, 0.9412, 0.9373,  ..., 0.7961, 0.8000, 0.8039],
          [0.9294, 0.9294, 0.9333,  ..., 0.7412, 0.7451, 0.7451],
          [0.9216, 0.9216, 0.9255,  ..., 0.7255, 0.7255, 0.7255],
          ...,
          [0.8118, 0.8000, 0.7647,  ..., 0.6745, 0.6706, 0.6627],
          [0.7647, 0.7333, 0.6941,  ..., 0.6941, 0.6980, 0.6863],
          [0.7059, 0.6667, 0.6510,  ..., 0.7059, 0.7294, 0.7255]],

         [[0.9451, 0.9451, 0.9412,  ..., 0.8000, 0.8039, 0.8078],
          [0.9333, 0.9333, 0.9373,  ..., 0.7451, 0.7490, 0.7490],
          [0.9255, 0.9255, 0.9294,  ..., 0.7294, 0.7294, 0.7294],
          ...,
          [0.7569, 0.7569, 0.7216,  ..., 0.6510, 0.6431, 0.6353],
          [0.7098, 0.6784, 0.6510,  ..., 0.6706, 0.6706, 0.6588],
          [0.6510, 0.6118, 0.6078,  ..., 0.6824, 0.7020, 0.6980]],

         [[0.9647, 0.9647, 0.9608,  ..., 0.8157, 0.8196, 0.8235],
          [0.9529, 0.9529, 0.9569,  ..., 0.7608, 0.7647, 0.7647],
          [0.9451, 0.9451, 0.9490,  ..., 0